In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

## Histogramas
1. Leer las imágenes en escala de grises con OpenCV y visualizarlas.
2. Elegir el número de bins que se crea conveniente y grafique su histograma, compare los histogramas entre sí.

In [ ]:
IMAGE_ROOT_PATH = "imgs/"
IMAGES_FILENAMES = ["img1_tp.png", "img2_tp.png"]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=len(IMAGES_FILENAMES))
fig.set_size_inches(8, 8)
for ax_img, ax_hist, img_file in zip(axs[:][0], axs[:][1], IMAGES_FILENAMES):
    img = cv2.imread(IMAGE_ROOT_PATH + img_file, cv2.IMREAD_GRAYSCALE)
    ax_img.imshow(img, cmap="gray")
    ax_img.set_title(img_file)
    ax_hist.hist(img.ravel(), 256, [0, 256])

Se puede observar que los histogramas de ambas imágenes son iguales, por lo tanto, las tonalidades de la imágen de la flor (derecha) coinciden con las del gradiente (izquierda). Es decir, si se ordenan los pixeles de negro (0, 0, 0) a blanco (255, 255, 255) en la imagen de la flor se obtiene el gradiente.

El histograma es útil como feature si se desea entrenar un modelo de clasificación/detección ya que se puede asociar los distintos picos del mismo a áreas específicas de la imágen.

### Segmentación utilizando histogramas
Previsualización de imágen a analizar.

In [ ]:
image = cv2.cvtColor(cv2.imread(IMAGE_ROOT_PATH + "segmentacion.png"), cv2.COLOR_BGR2RGB)
plt.imshow(image)

Análisis de los histogramas para los canales RGB

In [ ]:
COLORS = ("r", "g", "b")
fig, axs = plt.subplots(nrows=2, ncols=3)
fig.set_size_inches(15, 5)
axs[0][0].set_title("Red channel")
axs[0][1].set_title("Green channel")
axs[0][2].set_title("Blue channel")
for i, (color, img_chn) in enumerate(zip(COLORS, cv2.split(img))):
    histr = cv2.calcHist([img], [i], None, [256], [0, 256])
    axs[1][i].plot(histr, color=color)
    axs[1][i].set_xlim([0, 256])
    axs[1][i].grid()
    axs[0][i].imshow(img_chn, cmap="grey")

Segmentación de la imágen en cielo, agua y tierra (rocas).

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3)
fig.set_size_inches(15, 5)
axs[0].set_title("Water mask")
axs[1].set_title("Sky mask")
axs[2].set_title("Earth mask")

img = cv2.cvtColor(cv2.imread(IMAGE_ROOT_PATH + "segmentacion.png"), cv2.COLOR_BGR2RGB)
img_red, img_green, img_blue = cv2.split(img)

sea_fom_mask = (img_green > 200) & (img_red > 200) & (img_blue > 200)
mask_earth = (img_green < 50) | (img_blue < 50)
mask_sky = (img_blue > 150) & ~sea_fom_mask
mask_water = (img_green > 60) & (img_green < 110) & (img_blue > 80) & (img_blue < 120) | sea_fom_mask
axs[0].imshow(cv2.bitwise_and(img, img, mask=mask_water.astype(np.uint8) * 255))
axs[1].imshow(cv2.bitwise_and(img, img, mask=mask_sky.astype(np.uint8) * 255))
axs[2].imshow(cv2.bitwise_and(img, img, mask=mask_earth.astype(np.uint8) * 255))